In [2]:
# Library
from base import S1_CENTERS_FILE
from pandas import read_csv, DataFrame
from tqdm import tqdm

# Read Data
triangles:DataFrame = read_csv(S1_CENTERS_FILE)


In [3]:
# #############################################################################
# Algo
# #############################################################################
from sklearn.cluster import MeanShift, estimate_bandwidth
from numpy import percentile, array
from pandas import DataFrame, Series

# Get Just the centers
triangle_centers = triangles[['centroid_x','centroid_y','centroid_z']].as_matrix()

# Compute bandwidth for clustering with MeanShift
bandwidth = estimate_bandwidth(triangle_centers, quantile=0.025, n_samples=200)

# clustering with MeanShift
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(triangle_centers)

# Grab Cluster Labels
triangles['ms_middle'] = Series(ms.labels_, index=triangles.index)

# Attach Cluster Centers
centers_and_clusters = triangles.merge( 
    DataFrame(ms.cluster_centers_, columns=['ms_x','ms_y','ms_z']), 
    left_on='ms_middle', 
    right_index=True 
)

In [4]:
# Compute Distances
cx = centers_and_clusters['centroid_x']
cy = centers_and_clusters['centroid_y']
cz = centers_and_clusters['centroid_z']

msx = centers_and_clusters['ms_x']
msy = centers_and_clusters['ms_y']
msz = centers_and_clusters['ms_z']

# Get Ready to plot
size = DataFrame({
    'ms_dist': (((cx-msx)**2) + ((cy-msy)**2)  + ((cz-msz)**2))**0.5,
    'ms_middle': ms.labels_
}).groupby(['ms_middle'])['ms_dist'].max()

In [15]:
colors[size > percentile(size, q = 75)] = 'red'
size = 1000*size/max(size)


7.828923720082757